In [54]:
import jieba.posseg as pseg
import codecs
from gensim import corpora,models,similarities
import pandas as pd
import numpy as np

In [143]:
#构建停用词表
#def stopwords_list(stopwords):
 #   stopwords_list = [words.strip() for words in stopwords]
  #  stop_flag = ['x', 'c', 'u','d', 'p', 't', 'uj', 'm', 'f', 'r']
   # return stopwords_list,stop_flag

#对句子分词、去停用词
def tokenization(str_text):
    res = []
    words = pseg.cut(str_text)
    #分词
    stopwords = ['啊','的','得']
    stopwords_list = [words.strip() for words in stopwords]
    stop_flag = ['x', 'c', 'u','d', 'p', 't', 'uj', 'm', 'f', 'r']
    #stopwordslist,stop_flag = stopwords_list('/Users/taoxudong/huli/test/stop_words.txt')
    #分词特性？中文对应着stop_flag的含义。？？？？？？
    for word,flag in words:
        if word not in stopwords_list and flag not in stop_flag:
            if word != '\t':
                res.append(word)
    return res

In [187]:
def similarity_searchStr(search_str,get_search_list):
    str_list = get_search_list
    corpus = []
    for each in str_list:
        corpus.append(tokenization(each['title']))
    #建立词袋模型，生成字典和向量语料
    #print(corpus)
    dictionary = corpora.Dictionary(corpus)
    doc_vectors = [dictionary.doc2bow(text) for text in corpus]
    
    #建立tf-idf模型
    tfidf = models.TfidfModel(doc_vectors)
    tfidf_vectors = tfidf[doc_vectors]
    #len(tfidf_vectors) = 文本数  len(tfidf_vectors[0] == 第一个文本中的词数量)
    #print(len(tfidf_vectors))
    query = tokenization(search_str)
    #print(query)
    query_bow = dictionary.doc2bow(query)
    index = similarities.MatrixSimilarity(tfidf_vectors)
    sims = index[query_bow]
    #print(list(enumerate(sims)))
    max_index = list(sims).index(max(sims))
    #res = []
    res_dict = {}
    res_dict.update(search_str = search_str)
    if max(sims) == 0:
        res_dict.update(url = None)
    else:
        res_dict.update(url = get_search_list[max_index]['url'])
        #res.append(get_search_list[max_index]['title'])
        #res.append(get_search_list[max_index]['url'])
    return res_dict

In [188]:
get_search_list = [{'title': '北大方正软件技术学院', 'url': 'https://baike.baidu.com/item/%E5%8C%97%E5%A4%A7%E6%96%B9%E6%AD%A3%E8%BD%AF%E4%BB%B6%E6%8A%80%E6%9C%AF%E5%AD%A6%E9%99%A2/8593474'}, {'title': '刘海涛(北京北大方正软件技术学院财经分院会计讲师)', 'url': 'https://baike.baidu.com/item/%E5%88%98%E6%B5%B7%E6%B6%9B/20178624'}, {'title': '黄根隆', 'url': 'https://baike.baidu.com/item/%E9%BB%84%E6%A0%B9%E9%9A%86'}, {'title': '民办高校', 'url': 'https://baike.baidu.com/item/%E6%B0%91%E5%8A%9E%E9%AB%98%E6%A0%A1/809739'}, {'title': '全国大学生公益社团联盟', 'url': 'https://baike.baidu.com/item/%E5%85%A8%E5%9B%BD%E5%A4%A7%E5%AD%A6%E7%94%9F%E5%85%AC%E7%9B%8A%E7%A4%BE%E5%9B%A2%E8%81%94%E7%9B%9F'}, {'title': '动漫设计与制作专业', 'url': 'https://baike.baidu.com/item/%E5%8A%A8%E6%BC%AB%E8%AE%BE%E8%AE%A1%E4%B8%8E%E5%88%B6%E4%BD%9C%E4%B8%93%E4%B8%9A/6504338'}, {'title': '全国民办普通高等学校', 'url': 'https://baike.baidu.com/item/%E5%85%A8%E5%9B%BD%E6%B0%91%E5%8A%9E%E6%99%AE%E9%80%9A%E9%AB%98%E7%AD%89%E5%AD%A6%E6%A0%A1/12778312'}, {'title': '三校高职类', 'url': 'https://baike.baidu.com/item/%E4%B8%89%E6%A0%A1%E9%AB%98%E8%81%8C%E7%B1%BB/7898123'}]

In [189]:
search_str = '北京北大方正软件职业技术学院'

In [190]:
a = '鄂城钢铁厂职工大学'

In [191]:
b = [{'title': '成人高校(成人高等学校)', 'url': 'https://baike.baidu.com/item/%E6%88%90%E4%BA%BA%E9%AB%98%E6%A0%A1/141665'}, {'title': '中国成人院校', 'url': 'https://baike.baidu.com/item/%E4%B8%AD%E5%9B%BD%E6%88%90%E4%BA%BA%E9%99%A2%E6%A0%A1'}, {'title': '全国成人高等学校', 'url': 'https://baike.baidu.com/item/%E5%85%A8%E5%9B%BD%E6%88%90%E4%BA%BA%E9%AB%98%E7%AD%89%E5%AD%A6%E6%A0%A1/12763624'}, {'title': '成人高等学校', 'url': 'https://baike.baidu.com/item/%E6%88%90%E4%BA%BA%E9%AB%98%E7%AD%89%E5%AD%A6%E6%A0%A1/3459368'}, {'title': '陈敏(深圳市龙岗区建设局(住宅局)局长)', 'url': 'https://baike.baidu.com/item/%E9%99%88%E6%95%8F/7323165'}, {'title': '百科教育与科学研究组维护词条列表(湖北学校篇)', 'url': 'https://baike.baidu.com/item/%E7%99%BE%E7%A7%91%E6%95%99%E8%82%B2%E4%B8%8E%E7%A7%91%E5%AD%A6%E7%A0%94%E7%A9%B6%E7%BB%84%E7%BB%B4%E6%8A%A4%E8%AF%8D%E6%9D%A1%E5%88%97%E8%A1%A8%EF%BC%88%E6%B9%96%E5%8C%97%E5%AD%A6%E6%A0%A1%E7%AF%87%EF%BC%89'}]

In [192]:
res_titile1 = similarity_searchStr(a,b)

In [193]:
res_titile1

{'search_str': '鄂城钢铁厂职工大学', 'url': None}

In [194]:
res_url1

'https://baike.baidu.com/item/%E6%88%90%E4%BA%BA%E9%AB%98%E6%A0%A1/141665'

In [195]:
res_titile = similarity_searchStr(search_str,get_search_list)

In [196]:
res_titile

{'search_str': '北京北大方正软件职业技术学院',
 'url': 'https://baike.baidu.com/item/%E5%8C%97%E5%A4%A7%E6%96%B9%E6%AD%A3%E8%BD%AF%E4%BB%B6%E6%8A%80%E6%9C%AF%E5%AD%A6%E9%99%A2/8593474'}